In [ ]:
import os
from collections import defaultdict
import statistics

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tools.utils.settings import DefaultPath as defpath
from tools.utils.utils import get_mongodb_collections

In [ ]:
test_name = 'main_test'

In [ ]:
small = True
mongoclient, collections = get_mongodb_collections(small)

In [ ]:
test_dir = defpath.data_path.tests + '/' + test_name
results_dir = test_dir + '/results/extracted'

In [ ]:
solvers = [('josie', 'set'), ('josie', 'bag'), ('lshforest', 'set'), ('lshforest', 'bag')]
q = '10'

results = pd.read_csv(f'{results_dir}/final_results_q{q}.csv')

In [ ]:
results

In [ ]:
results['difference_overlap'] = results['algorithm_overlap'] - results['sloth_overlap']
results['algorithm_overlap_norm'] = results['algorithm_overlap'] / (results['sloth_overlap'] + 1)

## I vari metodi per caso ritornano risultati che in realtà non hanno overlap reale?

JOSIE no perché esatto, LSHForest può sbagliare (raramente)

In [ ]:
nullity_threshold = 0   # if the actual overlap between two tables isprint(next(resgroup)) under this threshold, the result is considered bad

x = []
for am, am_group in results.groupby(by=["algorithm", "mode"]):
    for query_id, q_group in am_group.groupby(by=["query_id"]):
        cnt = ((q_group['sloth_overlap'] == 0)).sum()
        num_query_results = q_group.count().values.tolist()[0]
        x.append([am[0], am[1], query_id[0], num_query_results, cnt, cnt / num_query_results])

x = pd.DataFrame(x, columns=['algorithm', 'mode', 'query_id', 'query_size', 'zero_overlap_cnt', 'zero_overlap_ratio'])

null_ratio_pivot = pd.pivot_table(x, values=['zero_overlap_ratio'], index=['algorithm', 'mode'], aggfunc=['mean', 'std', 'min', 'max'])
null_ratio_pivot

## Algorithm vs True Overlap

In [ ]:
data = [(am[0], am[1], group[(group['sloth_overlap'] != 0) & (group['difference_overlap'] != 0)]) for am, group in results.groupby(by=['algorithm', 'mode'])]

In [ ]:
fig, ax = plt.subplots(1, 1, sharey='row', figsize=(15, 5))
xmin, xmax, step = -200, 300, 10

ax.hist([d[2]['difference_overlap'] for d in data], 
         bins=np.arange(xmin, xmax, step), alpha=0.8, 
         label=[f'{a}-{m}' for a, m, _ in data],
         align='mid')
ax.set_xlim(xmin, xmax)
ax.set_xticks(np.arange(xmin, xmax, step))
ax.set_yscale('log')
ax.tick_params(axis='x', rotation=45)
ax.grid()
ax.set_xlabel('ALGORITHM overlap - SLOTH overlap')
ax.set_ylabel('frequency')

plt.legend()
plt.show()

In [ ]:
data = [(am[0], am[1], group[(group['sloth_overlap'] != 0) & (group['difference_overlap'] != 0)]) for am, group in results.groupby(by=['algorithm', 'mode'])]

In [ ]:
fig, ax = plt.subplots(1, 1, sharey='row', figsize=(15, 5))
xmin, xmax, step = 0, 9.6, 0.2

ax.hist([d[2]['algorithm_overlap_norm'] for d in data], 
         bins=np.arange(xmin, xmax, step), alpha=0.8, 
         label=[f'{a}-{m}' for a, m, _ in data],
         align='mid')
ax.set_xlim(xmin, xmax)
ax.set_xticks(np.arange(xmin, xmax, step))
ax.grid()
ax.set_xlabel('ALGORITHM_overlap / (SLOTH_overlap + 1)')
ax.set_ylabel('frequency')
ax.set_yscale('log')
ax.tick_params(axis='x', rotation=45)
plt.legend()
plt.show()

## Creating Silver Standard

In [ ]:
silver_standard = defaultdict(set)

results_ids = results.convert_dtypes().groupby(by='query_id')[['result_id', 'sloth_overlap']]

for query_id, ids_overlaps in results_ids:
    for i in ids_overlaps.values:
        _id, _overlap = i
        silver_standard[query_id].add((_id, _overlap))

for query_id in silver_standard.keys():
    silver_standard[query_id] = sorted(list(silver_standard[query_id]), key=lambda x: x[1], reverse=True)

## Precision at p - $P@p$
Corresponds to the number of relevant results among the top $p$ retrieved documents. Fails to take into account the positions of the relevant documents among the top $p$.Another shortcoming is that on a query with fewer relevant results than $p$, even a perfect system will have a score less than 1.


In [ ]:
k_precisions = [1, 3, 5, 10]

precision_at_k_results = []

for query_id in silver_standard.keys():
    qss = [x[1] for x in silver_standard[query_id]]
    avg_overlap = round(statistics.mean(qss), 3)
    stdev_overlap = round(statistics.stdev(qss))

    for (algorithm, mode), data in results.groupby(by=["algorithm", "mode"]):
        ids = data[data['query_id'] == query_id]['result_id'].values.tolist()
        for k_precision in k_precisions:
            real_topk = [x[0] for x in silver_standard[query_id][:k_precision]]
            precision_at_k = set(real_topk).intersection(ids)
            
            precision_at_k_results.append([query_id, len(qss), avg_overlap, stdev_overlap, algorithm, mode, k_precision, len(precision_at_k)])

columns = [
    'query_id',
    'silver_std_size',
    'silver_std_ov_mean',
    'silver_std_ov_stdev',
    'algorithm',
    'mode',
    'k',
    'precision_at_k'
]

precision_at_k_results = pd.DataFrame(precision_at_k_results, columns=columns)
precision_at_k = precision_at_k_results.sort_values(by=['silver_std_size', 'query_id'], ascending=False)
# precision_at_k

In [ ]:
patk_pivot = pd.pivot_table(precision_at_k, values=['precision_at_k'], index=['algorithm', 'mode'], columns=['k'], aggfunc=['mean', 'std', 'max'])
patk_pivot

In [ ]:
for row, label in zip(patk_pivot['mean', 'precision_at_k'].values, patk_pivot.index):
    plt.plot([1, 3, 5, 10], row, 'o-', label=f'{label[0]}-{label[1]}')
plt.xticks([1, 3, 5, 10], [1, 3, 5, 10])
plt.xlabel('p')
plt.ylabel('mean precision@p')
plt.legend()
plt.grid()

## Normalized Discontinued Cumulative Gain - $nDCG@p$

Search result lists vary in length depending on the query. Comparing a search engine's performance from one query to the next cannot be consistently achieved using DCG alone, so the cumulative gain at each position for a chosen value of $ p $ should be normalized across queries. This is done by sorting all relevant documents in the corpus by their relative relevance, producing the maximum possible DCG through position $p$, also called Ideal DCG (IDCG) through that position. For a query, the normalized discounted cumulative gain, or nDCG, is computed as: 

$ nDCG_{p} = {DCG_{p} \over IDCG_{p}} $

where $ IDCG_{p} $ is the ideal discounted cumulative gain,

$ IDCG_{p} = \sum_{i=1}^{|REL_{p}|} {2^{rel_{i}} - 1 \over log_{2}(i + 1)}$

where $ REL_{p} $ represents the list of relevant documents (ordered by their relevance) in the corpus up to position $p$

In [ ]:
from math import log2

def ndcg_at_p(true_relevances, scores, p):
    p = min(p, len(true_relevances), len(scores))
    if p <= 0: # because computing nDCG is meaningful only if there is more than one document 
        return 0, 1
    idcg = sum(rel / log2(i + 1) for i, rel in enumerate(true_relevances[:p], start=1))
    dcg = sum(rel / log2(i + 1) for i, rel in enumerate(scores[:p], start=1))
    if idcg < dcg:
        raise Exception()

    return dcg / idcg, p 

In [ ]:
def get_nDCG_p(silver_standard:defaultdict[int:list[tuple[int,int]]], results:pd.DataFrame, *p):
    """ p values are assumed positive """
    ndcg_res = []
    for query_id in silver_standard:
        true_relevances = [x[1] for x in silver_standard[query_id]]
        max_silver_standard = true_relevances[0]

        for (algorithm, mode), data in results.groupby(by=['algorithm', 'mode']):
            r = data[data['query_id'] == query_id][['result_id', 'sloth_overlap']]
            result_relevances = [min(max_silver_standard, x[1]) for x in r.values.tolist()]
            for _p in p:
                ndcg, _actual_p = ndcg_at_p(true_relevances, result_relevances, _p)
                if query_id == 29938 and algorithm == 'josie' and mode == 'bag':
                    print(ndcg, _p, true_relevances[:_p], result_relevances[:_p])
                ndcg_res.append([query_id, len(true_relevances), algorithm, mode, _p, _p - _actual_p, ndcg])
    return ndcg_res

In [ ]:
res = get_nDCG_p(silver_standard, results, 1, 3, 5, 10)

In [ ]:
df = pd.DataFrame(res, columns=['query_id', 'silver_standard_size', 'algorithm', 'mode', 'p', 'missing_p', 'ndcg_p'])

In [ ]:
silver_standard_size_threshold = 5
df = df[df['silver_standard_size'] >= silver_standard_size_threshold]

In [ ]:
ndcg_pivot = df.pivot_table(index=['algorithm', 'mode'], columns=['p'], values=['ndcg_p', 'missing_p'], aggfunc=['mean', 'max']).convert_dtypes()
ndcg_pivot

In [ ]:
ndcg_pivot = df.pivot_table(index=['algorithm', 'mode'], columns=['p'], values=['ndcg_p'], aggfunc=['mean', 'max']).convert_dtypes()
ndcg_pivot

In [ ]:
for row, label in zip(ndcg_pivot['mean', 'ndcg_p'].values, ndcg_pivot.index):
    plt.plot([1, 3, 5, 10], row, 'o-', label=f'{label[0]}-{label[1]}')
plt.xticks([1, 3, 5, 10], [1, 3, 5, 10])
plt.legend()
plt.xlabel("p")
plt.ylabel("mean nDCG@p")
plt.grid()

## Analisi tempo

In [ ]:
import itertools


x = [ 
    (
        algorithm, mode, nsamples, 
        pd.read_csv(test_dir + f'/results/base/a{algorithm}_m{mode}_k10_q{nsamples}.csv')['duration'].sum(),  
        pd.read_csv(test_dir + f'/results/base/a{algorithm}_m{mode}_k10_q{nsamples}.csv')['duration'].mean()
    )
    for algorithm, mode, nsamples in itertools.product(['josie', 'lshforest'], ['set', 'bag'], ["10"])
]

pd.DataFrame(x, columns=['algorithm', 'mode', 'num query', 'top-K time total (s)', 'top-K time mean (s)'])